### Set standard imports

In [ ]:
import matplotlib,lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib qt
import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)
import import_library as il
reload(il)

In [ ]:
#ta=pf.TA('gui',conversion_function=il.streak_Lund,shift_times_by=-552)
ta=pf.TA('FeCM02-266nm-4mw-QB390-t6-G63-w450-s150-556ms-E100.dat',conversion_function=il.streak_Lund,path='Data')

In [ ]:
# Coarse adjustment
ta.intensity_range=[0,ta.ds.mean().describe()['75%']*2]
ta.timelimits=[ta.ds.index.values.min(),ta.ds.index.values.max()]
ta.Cor_Chirp(shown_window=ta.timelimits,just_shift=True,cmap=pf.cm.jet)
if 0: # set to 1 for fine adjustment
    shown_window=np.array([-50,50])+ta.fitcoeff[-1]
    ta.Man_Chirp(shown_window=shown_window,just_shift=True,cmap=pf.cm.jet)

In [ ]:
plt.close('all')
ta.bordercut=[390,480]
ta.rel_wave=[400,410,420,460,480]
ta.rel_time=[250,600,750,1000,1750]
ta.timelimits=[1,1400]
ta.wave_nm_bin=None
ta.wavelength_bin=20
ta.log_scale=True
ta.intensity_range=[2000,40000]
ta.Plot_RAW(range(3),scale_type='linear')

### Perform fast fitting using independent exponential decay (first order)

In [ ]:
ta.wave_nm_bin=2; 
plt.close('all')                                             # Close old spectra
ta.mod='exponential'                                         # Choose the model 'exponential', 'consecutive' and 'fast_consecutive'are build in
par=lmfit.Parameters()                                       # create empty parameter object

par.add('k0',value=1/10,vary=True,min=1/100)                                # Add second rate component
par.add('k1',value=1/500,vary=True)                                # Add one rate component
par.add('t0',value=0.28,vary=False)                       # Allow the arrival time to adjust (for all)
par.add('resolution',value=18.4,vary=False)       # Allow the instrument response to adjust (for all)
#par.add('infinite')                                                # Keyword for an non decaying component
par.add('background')                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect
ta.log_fit=False                                               # fit in linear or log scale
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global(fit_chirp=False)                                 # trigger fitting

ta.error_matrix_amplification=10;                              # Choose how much stronger should error be plotted       
ta.log_scale=False                                             # 2D plots in linear or log scale
ta.Plot_fit_output([0,1,2,4])                                 # plot the fit output

### Perform Error analysis and calculate confidence interval

In [ ]:
ta.par=ta.par_fit #write the best results back as starting parameter
ta.Fit_Global(confidence_level=0.66)

In [ ]:
plt.close('all')
ta.mod='consecutive'             # very quick fit using independent exponential followed by one round of sequential
ta.par=ta.par_fit                # copy fitted parameter into a new fit (e.g. different model)
ta.Fit_Global()   # The iterative chrip is best optimised using 'exponential'
ta.Plot_fit_output([0,1,2,4])     # plot the fit output